In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'whitegrid')
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
import scipy.stats as stats
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from scipy.stats import randint as sp_randint, uniform as sp_uniform
import lightgbm as lgb
from xgboost import XGBRFRegressor, XGBRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA

In [18]:
pd.set_option('display.max_columns', None)

In [33]:
train = pd.read_csv('train_dummy_final.csv')
train.head()

,Year,Sales,Events,Federal_Holidays,Nominal_GDP,Real_GDP,CPI,Unemp_Rate,Rate_CreditCard,Rate_PersonalLoan,Wages,Cotton_Price,Change,Upland_Planted,Upland_Harvested,Yield_Harvested,Production,Mill_Use,Exports,Temp_High,Temp_Avg,Temp_Low,Dew_High,Dew_Avg,Dew_Low,Humidity_High,Humidity_Avg,Humidity_Low,Sea_High,Sea_Avg,Sea_Low,Visibility_High,Visibility_Avg,Visibility_Low,Wind_Low,Wind_Avg,Wind_High,Precip,Fog,"Fog , Rain","Fog , Rain , Snow","Fog , Snow",Normal,Rain,"Rain , Snow",Snow,Thunderstorm,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,ProductCategory_OtherClothing,ProductCategory_WomenClothing
0,-5,1755.0,0.0,3.0,14421.752895,14407.053343,233.402,7.8,12.03,11.44,22.05,0.5770,4.02,9.296,7.559,799,12.589,4.17,11.55,0.741935,-2.096774,-5.322581,-6.096774,-9.903226,-14.000000,71.774194,57.354839,42.322581,1021.290323,1015.967742,1010.516129,16.0,14.000000,11.451613,27.000000,12.000000,39.322581,2.445806,0.0,0.0,2.0,2.0,21.0,3.0,0.0,3.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,-5,524.0,0.0,3.0,14421.752895,14407.053343,233.402,7.8,12.03,11.44,22.05,0.5770,4.02,9.296,7.559,799,12.589,4.17,11.55,0.741935,-2.096774,-5.322581,-6.096774,-9.903226,-14.000000,71.774194,57.354839,42.322581,1021.290323,1015.967742,1010.516129,16.0,14.000000,11.451613,27.000000,12.000000,39.322581,2.445806,0.0,0.0,2.0,2.0,21.0,3.0,0.0,3.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-5,936.0,0.0,3.0,14421.752895,14407.053343,233.402,7.8,12.03,11.44,22.05,0.5770,4.02,9.296,7.559,799,12.589,4.17,11.55,0.741935,-2.096774,-5.322581,-6.096774,-9.903226,-14.000000,71.774194,57.354839,42.322581,1021.290323,1015.967742,1010.516129,16.0,14.000000,11.451613,27.000000,12.000000,39.322581,2.445806,0.0,0.0,2.0,2.0,21.0,3.0,0.0,3.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,-5,1729.0,1.0,1.0,14389.200466,14366.176571,234.663,8.3,12.97,11.05,22.22,0.5521,-4.32,9.296,7.559,799,12.589,3.87,11.10,6.964286,2.785714,-1.642857,-2.464286,-6.392857,-10.571429,69.535714,54.000000,38.107143,1021.714286,1017.071429,1012.464286,16.0,15.178571,13.285714,27.928571,12.214286,41.642857,0.782500,0.0,0.0,0.0,1.0,22.0,4.0,1.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,-5,496.0,1.0,1.0,14389.200466,14366.176571,234.663,8.3,12.97,11.05,22.22,0.5521,-4.32,9.296,7.559,799,12.589,3.87,11.10,6.964286,2.785714,-1.642857,-2.464286,-6.392857,-10.571429,69.535714,54.000000,38.107143,1021.714286,1017.071429,1012.464286,16.0,15.178571,13.285714,27.928571,12.214286,41.642857,0.782500,0.0,0.0,0.0,1.0,22.0,4.0,1.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
test = pd.read_csv('test_dummy_final.csv')
test.head()

,Year,Events,Federal_Holidays,Nominal_GDP,Real_GDP,CPI,Unemp_Rate,Rate_CreditCard,Rate_PersonalLoan,Wages,Cotton_Price,Change,Upland_Planted,Upland_Harvested,Yield_Harvested,Production,Mill_Use,Exports,Temp_High,Temp_Avg,Temp_Low,Dew_High,Dew_Avg,Dew_Low,Humidity_High,Humidity_Avg,Humidity_Low,Sea_High,Sea_Avg,Sea_Low,Visibility_High,Visibility_Avg,Visibility_Low,Wind_Low,Wind_Avg,Wind_High,Precip,Fog,"Fog , Rain","Fog , Rain , Snow","Fog , Snow",Normal,Rain,"Rain , Snow",Snow,Thunderstorm,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,ProductCategory_OtherClothing,ProductCategory_WomenClothing
0,0,0.0,3.0,16955.608626,15705.972667,259.596,6.6,11.85,10.22,24.35,0.9096,3.97,10.206,7.465,807,12.551,3.58,9.75,1.733333,-1.766667,-5.700000,-5.166667,-9.400000,-13.833333,75.033333,59.433333,43.900000,1025.033333,1019.200000,1012.233333,15.90,13.866667,10.633333,22.700000,10.10,38.866667,2.000333,0.0,3.0,0.0,1.0,20.0,2.0,1.0,3.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0.0,3.0,16955.608626,15705.972667,259.596,6.6,11.85,10.22,24.35,0.9096,3.97,10.206,7.465,807,12.551,3.58,9.75,1.733333,-1.766667,-5.700000,-5.166667,-9.400000,-13.833333,75.033333,59.433333,43.900000,1025.033333,1019.200000,1012.233333,15.90,13.866667,10.633333,22.700000,10.10,38.866667,2.000333,0.0,3.0,0.0,1.0,20.0,2.0,1.0,3.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0.0,3.0,16955.608626,15705.972667,259.596,6.6,11.85,10.22,24.35,0.9096,3.97,10.206,7.465,807,12.551,3.58,9.75,1.733333,-1.766667,-5.700000,-5.166667,-9.400000,-13.833333,75.033333,59.433333,43.900000,1025.033333,1019.200000,1012.233333,15.90,13.866667,10.633333,22.700000,10.10,38.866667,2.000333,0.0,3.0,0.0,1.0,20.0,2.0,1.0,3.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,1.0,1.0,17028.586348,15762.598920,259.019,6.7,11.83,10.09,24.58,0.9405,3.40,10.206,7.465,807,12.551,3.58,9.75,3.285714,0.285714,-3.107143,-4.178571,-7.821429,-11.678571,74.892857,58.714286,42.035714,1022.321429,1016.964286,1011.964286,15.75,12.357143,8.535714,25.178571,10.25,41.392857,4.972857,0.0,1.0,4.0,3.0,17.0,2.0,0.0,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,0,1.0,1.0,17028.586348,15762.598920,259.019,6.7,11.83,10.09,24.58,0.9405,3.40,10.206,7.465,807,12.551,3.58,9.75,3.285714,0.285714,-3.107143,-4.178571,-7.821429,-11.678571,74.892857,58.714286,42.035714,1022.321429,1016.964286,1011.964286,15.75,12.357143,8.535714,25.178571,10.25,41.392857,4.972857,0.0,1.0,4.0,3.0,17.0,2.0,0.0,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
combined = [train, test]

In [36]:
train.columns

Index(['Year', 'Sales', 'Events', 'Federal_Holidays', 'Nominal_GDP',
       'Real_GDP', 'CPI', 'Unemp_Rate', 'Rate_CreditCard', 'Rate_PersonalLoan',
       'Wages', 'Cotton_Price', 'Change', 'Upland_Planted', 'Upland_Harvested',
       'Yield_Harvested', 'Production', 'Mill_Use', 'Exports', 'Temp_High',
       'Temp_Avg', 'Temp_Low', 'Dew_High', 'Dew_Avg', 'Dew_Low',
       'Humidity_High', 'Humidity_Avg', 'Humidity_Low', 'Sea_High', 'Sea_Avg',
       'Sea_Low', 'Visibility_High', 'Visibility_Avg', 'Visibility_Low',
       'Wind_Low', 'Wind_Avg', 'Wind_High', 'Precip', 'Fog', 'Fog , Rain',
       'Fog , Rain , Snow', 'Fog , Snow', 'Normal', 'Rain', 'Rain , Snow',
       'Snow', 'Thunderstorm', 'Month_2', 'Month_3', 'Month_4', 'Month_5',
       'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11',
       'Month_12', 'ProductCategory_OtherClothing',
       'ProductCategory_WomenClothing'],
      dtype='object')

In [37]:
for i in combined:
    i['Holidays'] = i['Events'] + i['Federal_Holidays']
    i['GDP'] = (i['Nominal_GDP'] + i['Real_GDP']) / 2
    i['Interest_Rate'] = (i['Rate_CreditCard'] + i['Rate_PersonalLoan']) / 2
    i['Bad_Weather'] = i['Fog'] + i['Fog , Rain'] + i['Fog , Rain , Snow'] + i['Fog , Snow'] + i['Rain'] + i['Rain , Snow'] + i['Snow'] + i['Thunderstorm']

In [38]:
train = train.drop(['Events', 'Federal_Holidays', 'Nominal_GDP', 'Real_GDP', 'Rate_CreditCard', 'Rate_PersonalLoan', 
                'Upland_Planted', 'Upland_Harvested', 'Temp_High', 'Temp_Low', 'Dew_High', 'Dew_Low', 
                'Humidity_High', 'Humidity_Low', 'Sea_High', 'Sea_Low', 'Visibility_High', 'Visibility_Low',
               'Wind_Low', 'Wind_High', 'Fog , Rain', 'Fog , Rain , Snow', 'Fog , Snow', 'Rain', 'Rain , Snow', 'Snow',
               'Thunderstorm', 'Fog', 'Change'], axis = 1)

In [39]:
test = test.drop(['Events', 'Federal_Holidays', 'Nominal_GDP', 'Real_GDP', 'Rate_CreditCard', 'Rate_PersonalLoan', 
                'Upland_Planted', 'Upland_Harvested', 'Temp_High', 'Temp_Low', 'Dew_High', 'Dew_Low', 
                'Humidity_High', 'Humidity_Low', 'Sea_High', 'Sea_Low', 'Visibility_High', 'Visibility_Low',
               'Wind_Low', 'Wind_High', 'Fog , Rain', 'Fog , Rain , Snow', 'Fog , Snow', 'Rain', 'Rain , Snow', 'Snow',
               'Thunderstorm', 'Fog', 'Change'], axis = 1)

In [40]:
train.head()

,Year,Sales,CPI,Unemp_Rate,Wages,Cotton_Price,Yield_Harvested,Production,Mill_Use,Exports,Temp_Avg,Dew_Avg,Humidity_Avg,Sea_Avg,Visibility_Avg,Wind_Avg,Precip,Normal,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,ProductCategory_OtherClothing,ProductCategory_WomenClothing,Holidays,GDP,Interest_Rate,Bad_Weather
0,-5,1755.0,233.402,7.8,22.05,0.5770,799,12.589,4.17,11.55,-2.096774,-9.903226,57.354839,1015.967742,14.000000,12.000000,2.445806,21.0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.0,14414.403119,11.735,10.0
1,-5,524.0,233.402,7.8,22.05,0.5770,799,12.589,4.17,11.55,-2.096774,-9.903226,57.354839,1015.967742,14.000000,12.000000,2.445806,21.0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,14414.403119,11.735,10.0
2,-5,936.0,233.402,7.8,22.05,0.5770,799,12.589,4.17,11.55,-2.096774,-9.903226,57.354839,1015.967742,14.000000,12.000000,2.445806,21.0,0,0,0,0,0,0,0,0,0,0,0,1,0,3.0,14414.403119,11.735,10.0
3,-5,1729.0,234.663,8.3,22.22,0.5521,799,12.589,3.87,11.10,2.785714,-6.392857,54.000000,1017.071429,15.178571,12.214286,0.782500,22.0,1,0,0,0,0,0,0,0,0,0,0,0,1,2.0,14377.688519,12.010,6.0
4,-5,496.0,234.663,8.3,22.22,0.5521,799,12.589,3.87,11.10,2.785714,-6.392857,54.000000,1017.071429,15.178571,12.214286,0.782500,22.0,1,0,0,0,0,0,0,0,0,0,0,0,0,2.0,14377.688519,12.010,6.0


In [41]:
test.head()

,Year,CPI,Unemp_Rate,Wages,Cotton_Price,Yield_Harvested,Production,Mill_Use,Exports,Temp_Avg,Dew_Avg,Humidity_Avg,Sea_Avg,Visibility_Avg,Wind_Avg,Precip,Normal,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,ProductCategory_OtherClothing,ProductCategory_WomenClothing,Holidays,GDP,Interest_Rate,Bad_Weather
0,0,259.596,6.6,24.35,0.9096,807,12.551,3.58,9.75,-1.766667,-9.400000,59.433333,1019.200000,13.866667,10.10,2.000333,20.0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.0,16330.790646,11.035,10.0
1,0,259.596,6.6,24.35,0.9096,807,12.551,3.58,9.75,-1.766667,-9.400000,59.433333,1019.200000,13.866667,10.10,2.000333,20.0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,16330.790646,11.035,10.0
2,0,259.596,6.6,24.35,0.9096,807,12.551,3.58,9.75,-1.766667,-9.400000,59.433333,1019.200000,13.866667,10.10,2.000333,20.0,0,0,0,0,0,0,0,0,0,0,0,1,0,3.0,16330.790646,11.035,10.0
3,0,259.019,6.7,24.58,0.9405,807,12.551,3.58,9.75,0.285714,-7.821429,58.714286,1016.964286,12.357143,10.25,4.972857,17.0,1,0,0,0,0,0,0,0,0,0,0,0,1,2.0,16395.592634,10.960,11.0
4,0,259.019,6.7,24.58,0.9405,807,12.551,3.58,9.75,0.285714,-7.821429,58.714286,1016.964286,12.357143,10.25,4.972857,17.0,1,0,0,0,0,0,0,0,0,0,0,0,0,2.0,16395.592634,10.960,11.0


In [42]:
train.to_csv('new_train.csv', index = False)
test.to_csv('new_test.csv', index = False)

In [44]:
X_train = train.drop('Sales', 1)
y_train = train['Sales']
X_test = test

In [45]:
y_train_log = np.log(y_train)

In [46]:
ss = StandardScaler()
X_train_scaled = pd.DataFrame(ss.fit_transform(X_train), columns = X_train.columns)
X_test_scaled = pd.DataFrame(ss.transform(X_test), columns = X_test.columns)

In [47]:
X_train_scaled.head()

,Year,CPI,Unemp_Rate,Wages,Cotton_Price,Yield_Harvested,Production,Mill_Use,Exports,Temp_Avg,Dew_Avg,Humidity_Avg,Sea_Avg,Visibility_Avg,Wind_Avg,Precip,Normal,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,ProductCategory_OtherClothing,ProductCategory_WomenClothing,Holidays,GDP,Interest_Rate,Bad_Weather
0,-1.414214,-1.803351,-0.947473,-1.520639,-1.087254,0.125973,-1.051372,3.978623,-0.109905,-1.799562,-1.751915,-0.934442,0.084808,0.146405,1.257299,-0.480800,0.499525,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.707107,1.414214,0.636146,-1.392428,-0.078494,-0.301259
1,-1.414214,-1.803351,-0.947473,-1.520639,-1.087254,0.125973,-1.051372,3.978623,-0.109905,-1.799562,-1.751915,-0.934442,0.084808,0.146405,1.257299,-0.480800,0.499525,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.707107,-0.707107,0.636146,-1.392428,-0.078494,-0.301259
2,-1.414214,-1.803351,-0.947473,-1.520639,-1.087254,0.125973,-1.051372,3.978623,-0.109905,-1.799562,-1.751915,-0.934442,0.084808,0.146405,1.257299,-0.480800,0.499525,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,1.414214,-0.707107,0.636146,-1.392428,-0.078494,-0.301259
3,-1.414214,-1.635694,-0.392313,-1.260352,-1.151090,0.125973,-1.051372,2.165074,-0.365719,-1.234187,-1.357616,-1.472215,0.462549,1.447747,1.359193,-1.160801,0.852131,3.316625,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.707107,1.414214,-0.057831,-1.450561,0.445857,-1.718947
4,-1.414214,-1.635694,-0.392313,-1.260352,-1.151090,0.125973,-1.051372,2.165074,-0.365719,-1.234187,-1.357616,-1.472215,0.462549,1.447747,1.359193,-1.160801,0.852131,3.316625,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.707107,-0.707107,-0.057831,-1.450561,0.445857,-1.718947


In [48]:
X_test_scaled.head()

,Year,CPI,Unemp_Rate,Wages,Cotton_Price,Yield_Harvested,Production,Mill_Use,Exports,Temp_Avg,Dew_Avg,Humidity_Avg,Sea_Avg,Visibility_Avg,Wind_Avg,Precip,Normal,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,ProductCategory_OtherClothing,ProductCategory_WomenClothing,Holidays,GDP,Interest_Rate,Bad_Weather
0,2.12132,1.679290,-2.279856,2.000895,-0.234573,0.409856,-1.068776,0.411978,-1.133161,-1.761337,-1.695390,-0.601263,1.191060,-0.000818,0.353840,-0.66292,0.146919,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.707107,1.414214,0.636146,1.641943,-1.413205,-0.301259
1,2.12132,1.679290,-2.279856,2.000895,-0.234573,0.409856,-1.068776,0.411978,-1.133161,-1.761337,-1.695390,-0.601263,1.191060,-0.000818,0.353840,-0.66292,0.146919,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.707107,-0.707107,0.636146,1.641943,-1.413205,-0.301259
2,2.12132,1.679290,-2.279856,2.000895,-0.234573,0.409856,-1.068776,0.411978,-1.133161,-1.761337,-1.695390,-0.601263,1.191060,-0.000818,0.353840,-0.66292,0.146919,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,1.414214,-0.707107,0.636146,1.641943,-1.413205,-0.301259
3,2.12132,1.602575,-2.168824,2.353048,-0.155355,0.409856,-1.068776,0.411978,-1.133161,-1.523678,-1.518079,-0.716525,0.425879,-1.667587,0.425166,0.55232,-0.910898,3.316625,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.707107,1.414214,-0.057831,1.744549,-1.556210,0.053163
4,2.12132,1.602575,-2.168824,2.353048,-0.155355,0.409856,-1.068776,0.411978,-1.133161,-1.523678,-1.518079,-0.716525,0.425879,-1.667587,0.425166,0.55232,-0.910898,3.316625,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.707107,-0.707107,-0.057831,1.744549,-1.556210,0.053163


In [49]:
X_train_scaled.shape, X_test_scaled.shape

((180, 34), (36, 34))

### PCA

In [71]:
pca = PCA(19)

In [72]:
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [73]:
pca.explained_variance_ratio_.sum()

0.970181665801884

In [74]:
lr = LinearRegression()
lr.fit(X_train_pca, y_train_log)
lr.score(X_train_pca, y_train_log)

0.9781143081402032

### Modeling

In [ ]:
rfr = RandomForestRegressor(random_state = 0)

cbr = CatBoostRegressor(random_state = 0)

gbr = GradientBoostingRegressor(random_state = 0)

lgbmr = lgb.LGBMRegressor(random_state = 0)

xgbr = XGBRegressor(random_state = 0)
xgbrfr = XGBRFRegressor(random_state = 0)

votreg = VotingRegressor(estimators = [('rfr', rfr), ('cbr', cbr), ('gbr', gbr), 
                                       ('lgbmr', lgbmr), ('xgbr', xgbr), ('xgbrfr', xgbrfr)])

votreg2 = VotingRegressor(estimators = [('rfr', rfr), ('gbr', gbr), ('lgbmr', lgbmr)])

#### RF

In [ ]:
rfr.fit(X_train_pca, y_train_log)
rfr.score(X_train_pca, y_train_log)

##### Tuning RF

In [85]:
rfr = RandomForestRegressor(random_state = 0)
params = {'n_estimators' : sp_randint(50, 200),
         'max_features' : sp_randint(1, 20),
         'max_depth' : sp_randint(2, 50),
         'min_samples_leaf' : sp_randint(1, 25),
         'min_samples_split' : sp_randint(2, 25),
         'oob_score' : [False, True]}
rsearch = RandomizedSearchCV(rfr, param_distributions=params, cv = 5, n_jobs = -1, random_state = 0, n_iter = 100)
rsearch.fit(X_train_pca, y_train_log)
rsearch.best_params_

{'max_depth': 43,
 'max_features': 11,
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'n_estimators': 107,
 'oob_score': True}

#### Tuned RF

In [86]:
rfr = RandomForestRegressor(**rsearch.best_params_, random_state = 0)
rfr.fit(X_train_pca, y_train_log)
rfr.score(X_train_pca, y_train_log)

0.9931502204651264

#### CBR

In [87]:
cbr.fit(X_train_pca, y_train_log)
cbr.score(X_train_pca, y_train_log)

Learning rate set to 0.028763
0:	learn: 0.6591449	total: 2.1ms	remaining: 2.09s
1:	learn: 0.6451279	total: 6.84ms	remaining: 3.41s
2:	learn: 0.6302495	total: 9.28ms	remaining: 3.08s
3:	learn: 0.6179722	total: 12.1ms	remaining: 3.01s
4:	learn: 0.6049218	total: 14.4ms	remaining: 2.86s
5:	learn: 0.5943899	total: 16.5ms	remaining: 2.73s
6:	learn: 0.5832886	total: 18.4ms	remaining: 2.61s
7:	learn: 0.5720102	total: 20.5ms	remaining: 2.54s
8:	learn: 0.5614460	total: 22.3ms	remaining: 2.45s
9:	learn: 0.5484478	total: 23.8ms	remaining: 2.36s
10:	learn: 0.5381581	total: 25.6ms	remaining: 2.3s
11:	learn: 0.5294750	total: 27.6ms	remaining: 2.27s
12:	learn: 0.5194156	total: 29.5ms	remaining: 2.24s
13:	learn: 0.5090167	total: 31.4ms	remaining: 2.21s
14:	learn: 0.5012707	total: 33.2ms	remaining: 2.18s
15:	learn: 0.4923144	total: 35.2ms	remaining: 2.16s
16:	learn: 0.4822801	total: 37.2ms	remaining: 2.15s
17:	learn: 0.4734289	total: 40.3ms	remaining: 2.2s
18:	learn: 0.4651741	total: 44.4ms	remaining: 2

199:	learn: 0.0631013	total: 435ms	remaining: 1.74s
200:	learn: 0.0625460	total: 437ms	remaining: 1.74s
201:	learn: 0.0619676	total: 439ms	remaining: 1.74s
202:	learn: 0.0615747	total: 442ms	remaining: 1.74s
203:	learn: 0.0610547	total: 445ms	remaining: 1.74s
204:	learn: 0.0606486	total: 448ms	remaining: 1.74s
205:	learn: 0.0600859	total: 452ms	remaining: 1.74s
206:	learn: 0.0597090	total: 454ms	remaining: 1.74s
207:	learn: 0.0593834	total: 457ms	remaining: 1.74s
208:	learn: 0.0589022	total: 460ms	remaining: 1.74s
209:	learn: 0.0584722	total: 462ms	remaining: 1.74s
210:	learn: 0.0581279	total: 463ms	remaining: 1.73s
211:	learn: 0.0577289	total: 466ms	remaining: 1.73s
212:	learn: 0.0572429	total: 468ms	remaining: 1.73s
213:	learn: 0.0568992	total: 470ms	remaining: 1.73s
214:	learn: 0.0565823	total: 472ms	remaining: 1.72s
215:	learn: 0.0561863	total: 474ms	remaining: 1.72s
216:	learn: 0.0558590	total: 476ms	remaining: 1.72s
217:	learn: 0.0555085	total: 479ms	remaining: 1.72s
218:	learn: 

401:	learn: 0.0206945	total: 885ms	remaining: 1.32s
402:	learn: 0.0206285	total: 888ms	remaining: 1.31s
403:	learn: 0.0205402	total: 890ms	remaining: 1.31s
404:	learn: 0.0204985	total: 893ms	remaining: 1.31s
405:	learn: 0.0204092	total: 897ms	remaining: 1.31s
406:	learn: 0.0203130	total: 900ms	remaining: 1.31s
407:	learn: 0.0202529	total: 904ms	remaining: 1.31s
408:	learn: 0.0202247	total: 907ms	remaining: 1.31s
409:	learn: 0.0201346	total: 910ms	remaining: 1.31s
410:	learn: 0.0200448	total: 912ms	remaining: 1.31s
411:	learn: 0.0200061	total: 914ms	remaining: 1.3s
412:	learn: 0.0199566	total: 916ms	remaining: 1.3s
413:	learn: 0.0198736	total: 918ms	remaining: 1.3s
414:	learn: 0.0197829	total: 920ms	remaining: 1.3s
415:	learn: 0.0197228	total: 922ms	remaining: 1.29s
416:	learn: 0.0196331	total: 924ms	remaining: 1.29s
417:	learn: 0.0195382	total: 926ms	remaining: 1.29s
418:	learn: 0.0194246	total: 928ms	remaining: 1.29s
419:	learn: 0.0193440	total: 930ms	remaining: 1.28s
420:	learn: 0.01

603:	learn: 0.0104913	total: 1.34s	remaining: 880ms
604:	learn: 0.0104507	total: 1.35s	remaining: 879ms
605:	learn: 0.0104346	total: 1.35s	remaining: 876ms
606:	learn: 0.0104046	total: 1.35s	remaining: 874ms
607:	learn: 0.0103783	total: 1.35s	remaining: 872ms
608:	learn: 0.0103374	total: 1.35s	remaining: 870ms
609:	learn: 0.0103098	total: 1.36s	remaining: 868ms
610:	learn: 0.0102826	total: 1.36s	remaining: 866ms
611:	learn: 0.0102594	total: 1.36s	remaining: 864ms
612:	learn: 0.0102236	total: 1.36s	remaining: 862ms
613:	learn: 0.0101902	total: 1.37s	remaining: 859ms
614:	learn: 0.0101403	total: 1.37s	remaining: 857ms
615:	learn: 0.0101014	total: 1.37s	remaining: 854ms
616:	learn: 0.0100673	total: 1.37s	remaining: 852ms
617:	learn: 0.0100349	total: 1.38s	remaining: 850ms
618:	learn: 0.0099967	total: 1.38s	remaining: 849ms
619:	learn: 0.0099515	total: 1.38s	remaining: 846ms
620:	learn: 0.0099052	total: 1.38s	remaining: 844ms
621:	learn: 0.0098759	total: 1.38s	remaining: 841ms
622:	learn: 

830:	learn: 0.0054001	total: 1.81s	remaining: 369ms
831:	learn: 0.0053779	total: 1.81s	remaining: 366ms
832:	learn: 0.0053632	total: 1.82s	remaining: 364ms
833:	learn: 0.0053547	total: 1.82s	remaining: 362ms
834:	learn: 0.0053319	total: 1.82s	remaining: 360ms
835:	learn: 0.0053132	total: 1.82s	remaining: 358ms
836:	learn: 0.0052867	total: 1.83s	remaining: 356ms
837:	learn: 0.0052650	total: 1.83s	remaining: 354ms
838:	learn: 0.0052572	total: 1.84s	remaining: 353ms
839:	learn: 0.0052414	total: 1.84s	remaining: 351ms
840:	learn: 0.0052189	total: 1.84s	remaining: 348ms
841:	learn: 0.0052011	total: 1.85s	remaining: 346ms
842:	learn: 0.0051904	total: 1.85s	remaining: 344ms
843:	learn: 0.0051882	total: 1.85s	remaining: 342ms
844:	learn: 0.0051646	total: 1.85s	remaining: 340ms
845:	learn: 0.0051398	total: 1.85s	remaining: 338ms
846:	learn: 0.0051124	total: 1.86s	remaining: 336ms
847:	learn: 0.0050952	total: 1.86s	remaining: 334ms
848:	learn: 0.0050850	total: 1.86s	remaining: 332ms
849:	learn: 

0.9999752052431781

#### GBR

In [88]:
gbr.fit(X_train_pca, y_train_log)
gbr.score(X_train_pca, y_train_log)

0.9997199238581219

##### Tuning GBR

In [89]:
gbr = GradientBoostingRegressor(random_state = 0)

params = {'n_estimators' : sp_randint(50, 200),
          'max_features' : sp_randint(1, 20),
          'max_depth' : sp_randint(2, 50),
          'min_samples_leaf' : sp_randint(1, 25),
          'min_samples_split' : sp_randint(2, 25),
          'learning_rate' : sp_uniform(0.01, 0.1)}

rsearch_gbr = RandomizedSearchCV(gbr, param_distributions = params, n_jobs = -1,
                                 cv = 5, n_iter = 100, random_state = 0)

rsearch_gbr.fit(X_train_pca, y_train_log)
print(rsearch_gbr.best_params_)

{'learning_rate': 0.09817202123338396, 'max_depth': 48, 'max_features': 19, 'min_samples_leaf': 12, 'min_samples_split': 22, 'n_estimators': 160}


###### Tuned GBR

In [90]:
gbr = GradientBoostingRegressor(**rsearch_gbr.best_params_, random_state = 0)
gbr.fit(X_train_pca, y_train_log)
gbr.score(X_train_pca, y_train_log)

0.9997199238581219

#### LGBMR

In [91]:
lgbmr.fit(X_train_pca, y_train_log)
lgbmr.score(X_train_pca, y_train_log)

0.9950436807418938

##### Tuning LGBMR

In [92]:
lgbmr = lgb.LGBMRegressor(random_state = 0)

params = {'n_estimators' : sp_randint(50, 200),
         'max_depth' : sp_randint(1, 25),
         'learning_rate' : sp_uniform(0, 1)}

rsearch_lgbm = RandomizedSearchCV(lgbmr, param_distributions = params,
                                 cv = 5, n_iter = 100, n_jobs = -1, random_state = 0)

rsearch_lgbm.fit(X_train_pca, y_train_log)
print(rsearch_lgbm.best_params_)

{'learning_rate': 0.46924917399024213, 'max_depth': 6, 'n_estimators': 175}


###### Tuned LGBMR

In [94]:
lgbmr = lgb.LGBMRegressor(**rsearch_lgbm.best_params_, random_state = 0)
lgbmr.fit(X_train_pca, y_train_log)
gbr.score(X_train_pca, y_train_log)

0.9997199238581219

#### XGBR

In [95]:
xgbr.fit(X_train_pca, y_train_log)
xgbr.score(X_train_pca, y_train_log)

0.9999971768449365

#### XGBRFR

In [96]:
xgbrfr.fit(X_train_pca, y_train_log)
xgbrfr.score(X_train_pca, y_train_log)

0.9957777491264673

### VotReg

#### VotReg1

In [97]:
votreg.fit(X_train_pca, y_train_log)
votreg.score(X_train_pca, y_train_log)

Learning rate set to 0.028763
0:	learn: 0.6591449	total: 1.74ms	remaining: 1.74s
1:	learn: 0.6451279	total: 3.71ms	remaining: 1.85s
2:	learn: 0.6302495	total: 5.9ms	remaining: 1.96s
3:	learn: 0.6179722	total: 8.35ms	remaining: 2.08s
4:	learn: 0.6049218	total: 10.1ms	remaining: 2s
5:	learn: 0.5943899	total: 11.9ms	remaining: 1.97s
6:	learn: 0.5832886	total: 13.6ms	remaining: 1.92s
7:	learn: 0.5720102	total: 15.4ms	remaining: 1.91s
8:	learn: 0.5614460	total: 17.1ms	remaining: 1.88s
9:	learn: 0.5484478	total: 18.4ms	remaining: 1.82s
10:	learn: 0.5381581	total: 20ms	remaining: 1.8s
11:	learn: 0.5294750	total: 21.7ms	remaining: 1.78s
12:	learn: 0.5194156	total: 23.5ms	remaining: 1.78s
13:	learn: 0.5090167	total: 25.4ms	remaining: 1.79s
14:	learn: 0.5012707	total: 27.2ms	remaining: 1.78s
15:	learn: 0.4923144	total: 28.8ms	remaining: 1.77s
16:	learn: 0.4822801	total: 30.4ms	remaining: 1.75s
17:	learn: 0.4734289	total: 32.3ms	remaining: 1.76s
18:	learn: 0.4651741	total: 34.4ms	remaining: 1.77s

169:	learn: 0.0790643	total: 308ms	remaining: 1.5s
170:	learn: 0.0784739	total: 311ms	remaining: 1.51s
171:	learn: 0.0778374	total: 313ms	remaining: 1.51s
172:	learn: 0.0773223	total: 315ms	remaining: 1.51s
173:	learn: 0.0766557	total: 317ms	remaining: 1.5s
174:	learn: 0.0760587	total: 319ms	remaining: 1.5s
175:	learn: 0.0754764	total: 321ms	remaining: 1.5s
176:	learn: 0.0749907	total: 323ms	remaining: 1.5s
177:	learn: 0.0744324	total: 325ms	remaining: 1.5s
178:	learn: 0.0738335	total: 328ms	remaining: 1.5s
179:	learn: 0.0733286	total: 330ms	remaining: 1.5s
180:	learn: 0.0726329	total: 332ms	remaining: 1.5s
181:	learn: 0.0721803	total: 334ms	remaining: 1.5s
182:	learn: 0.0715375	total: 336ms	remaining: 1.5s
183:	learn: 0.0710670	total: 337ms	remaining: 1.5s
184:	learn: 0.0706775	total: 339ms	remaining: 1.49s
185:	learn: 0.0700000	total: 340ms	remaining: 1.49s
186:	learn: 0.0696195	total: 342ms	remaining: 1.49s
187:	learn: 0.0691641	total: 343ms	remaining: 1.48s
188:	learn: 0.0686188	to

340:	learn: 0.0267157	total: 615ms	remaining: 1.19s
341:	learn: 0.0265688	total: 616ms	remaining: 1.19s
342:	learn: 0.0263951	total: 618ms	remaining: 1.18s
343:	learn: 0.0262417	total: 620ms	remaining: 1.18s
344:	learn: 0.0260882	total: 622ms	remaining: 1.18s
345:	learn: 0.0259926	total: 624ms	remaining: 1.18s
346:	learn: 0.0258288	total: 627ms	remaining: 1.18s
347:	learn: 0.0256792	total: 628ms	remaining: 1.18s
348:	learn: 0.0256063	total: 630ms	remaining: 1.18s
349:	learn: 0.0255388	total: 633ms	remaining: 1.17s
350:	learn: 0.0254045	total: 635ms	remaining: 1.17s
351:	learn: 0.0253023	total: 638ms	remaining: 1.17s
352:	learn: 0.0251869	total: 640ms	remaining: 1.17s
353:	learn: 0.0250521	total: 642ms	remaining: 1.17s
354:	learn: 0.0249386	total: 644ms	remaining: 1.17s
355:	learn: 0.0248106	total: 646ms	remaining: 1.17s
356:	learn: 0.0246851	total: 648ms	remaining: 1.17s
357:	learn: 0.0245530	total: 649ms	remaining: 1.16s
358:	learn: 0.0244517	total: 651ms	remaining: 1.16s
359:	learn: 

564:	learn: 0.0118193	total: 1.07s	remaining: 826ms
565:	learn: 0.0117804	total: 1.07s	remaining: 824ms
566:	learn: 0.0117478	total: 1.08s	remaining: 822ms
567:	learn: 0.0116986	total: 1.08s	remaining: 820ms
568:	learn: 0.0116751	total: 1.08s	remaining: 818ms
569:	learn: 0.0116516	total: 1.08s	remaining: 816ms
570:	learn: 0.0116052	total: 1.08s	remaining: 814ms
571:	learn: 0.0115764	total: 1.08s	remaining: 812ms
572:	learn: 0.0115400	total: 1.09s	remaining: 810ms
573:	learn: 0.0115058	total: 1.09s	remaining: 808ms
574:	learn: 0.0114814	total: 1.09s	remaining: 807ms
575:	learn: 0.0114398	total: 1.09s	remaining: 805ms
576:	learn: 0.0114149	total: 1.09s	remaining: 803ms
577:	learn: 0.0113713	total: 1.1s	remaining: 802ms
578:	learn: 0.0113536	total: 1.1s	remaining: 800ms
579:	learn: 0.0113020	total: 1.1s	remaining: 798ms
580:	learn: 0.0112775	total: 1.1s	remaining: 796ms
581:	learn: 0.0112597	total: 1.1s	remaining: 794ms
582:	learn: 0.0112395	total: 1.11s	remaining: 793ms
583:	learn: 0.011

727:	learn: 0.0072004	total: 1.38s	remaining: 514ms
728:	learn: 0.0071684	total: 1.38s	remaining: 513ms
729:	learn: 0.0071399	total: 1.38s	remaining: 511ms
730:	learn: 0.0071137	total: 1.38s	remaining: 509ms
731:	learn: 0.0071013	total: 1.39s	remaining: 508ms
732:	learn: 0.0070903	total: 1.39s	remaining: 506ms
733:	learn: 0.0070607	total: 1.39s	remaining: 504ms
734:	learn: 0.0070309	total: 1.39s	remaining: 502ms
735:	learn: 0.0070076	total: 1.39s	remaining: 500ms
736:	learn: 0.0069788	total: 1.4s	remaining: 498ms
737:	learn: 0.0069476	total: 1.4s	remaining: 497ms
738:	learn: 0.0069396	total: 1.4s	remaining: 495ms
739:	learn: 0.0069241	total: 1.4s	remaining: 493ms
740:	learn: 0.0069149	total: 1.4s	remaining: 491ms
741:	learn: 0.0068923	total: 1.41s	remaining: 489ms
742:	learn: 0.0068844	total: 1.41s	remaining: 487ms
743:	learn: 0.0068649	total: 1.41s	remaining: 485ms
744:	learn: 0.0068546	total: 1.41s	remaining: 483ms
745:	learn: 0.0068359	total: 1.41s	remaining: 481ms
746:	learn: 0.006

888:	learn: 0.0045203	total: 1.68s	remaining: 210ms
889:	learn: 0.0045065	total: 1.68s	remaining: 208ms
890:	learn: 0.0045027	total: 1.69s	remaining: 206ms
891:	learn: 0.0044985	total: 1.69s	remaining: 204ms
892:	learn: 0.0044844	total: 1.69s	remaining: 203ms
893:	learn: 0.0044740	total: 1.69s	remaining: 201ms
894:	learn: 0.0044655	total: 1.69s	remaining: 199ms
895:	learn: 0.0044567	total: 1.7s	remaining: 197ms
896:	learn: 0.0044373	total: 1.7s	remaining: 195ms
897:	learn: 0.0044279	total: 1.7s	remaining: 193ms
898:	learn: 0.0044145	total: 1.7s	remaining: 191ms
899:	learn: 0.0043969	total: 1.7s	remaining: 189ms
900:	learn: 0.0043861	total: 1.71s	remaining: 188ms
901:	learn: 0.0043731	total: 1.71s	remaining: 186ms
902:	learn: 0.0043611	total: 1.71s	remaining: 184ms
903:	learn: 0.0043550	total: 1.71s	remaining: 182ms
904:	learn: 0.0043510	total: 1.71s	remaining: 180ms
905:	learn: 0.0043335	total: 1.71s	remaining: 178ms
906:	learn: 0.0043307	total: 1.72s	remaining: 176ms
907:	learn: 0.004

0.9990372889105383

#### VotReg2

In [98]:
votreg2.fit(X_train_pca, y_train_log)
votreg2.score(X_train_pca, y_train_log)

0.9978839284617298

### Test Prediction

In [99]:
pred = lgbmr.predict(X_test_pca)

#### Submission Data 

In [100]:
kaggle = pd.read_csv('Kaggle_Submission_Format.csv')

In [101]:
pred.shape

(36,)

In [102]:
kaggle['Sales(In ThousandDollars)'] = np.exp(pred)

In [103]:
kaggle.to_csv('Iteration_6_pca_19new_lgbmr', index = False)
kaggle.head()

,Year,Sales(In ThousandDollars)
0,1,2852.350528
1,2,645.033147
2,3,1074.247029
3,4,3037.750275
4,5,585.518075
